# Experiment Testing Adversaries

To do:

Option 1:
1. Find the training data
2. Load the training data
3. Select trajectories which contain scenario 1. 

The training data is very light on the optimal trajectories! My stupid sampling thing was almost certainly part of the problem. I think we should try removing it and seeing if the model will train. 

Option 2:
1. Synthesize scenario 1 forward passes somehow. 

Fixed Result:

4. Visualize a few. 
5. Write code to add adversaries. 
6. Plot left-right preference before/after. 

## Try to Load the Training Data and use that 

In [2]:
import sys 
import os 

sys.path.append("..")

from src.streamlit_app.environment import get_env_and_dt, preprocess_inputs
from src.environments.registration import register_envs
from src.decision_transformer.offline_dataset import TrajectoryDataset, one_hot_encode_observation
from torch.utils.data import DataLoader

register_envs()

# GET MODEL AND ENVIRONMENT
model_index = {
    # The original post, reproduced with 1/10th of the total training epochs
    "models/MiniGrid-MemoryS7FixedStart-v0/WorkingModel.pt": "MemoryDT",
    "models/MiniGrid-MemoryS7FixedStart-v0/MemoryGatedMLP.pt": "MemoryDTGatedMLP",
    "models/MiniGrid-Dynamic-Obstacles-8x8-v0/ReproduceOriginalPostShort.pt": "DynamicObstaclesDT_reproduction",
}

env, dt = get_env_and_dt("../models/MiniGrid-MemoryS7FixedStart-v0/WorkingModel.pt")

# GET TRAINING DATA 
training_trajectories = "../trajectories/MiniGrid-MemoryS7FixedStart-v0-Checkpoint11-VariedSamplingStrategies_T1_10k_T5_20k.gz"
dataset = TrajectoryDataset(
    training_trajectories, 
    max_len=9, 
    prob_go_from_end=1,
    preprocess_observations=one_hot_encode_observation)
s, a, r, d, rtg, ti, m = dataset[0]
print(s.shape)

Registering DynamicObstaclesMultiEnv-v0
Registering CrossingMultiEnv-v0
Registering Probe Envs


2023-10-01 17:50:13.934 
  command:

    streamlit run /Users/josephbloom/miniforge3/envs/decision_transformer_interpretability/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]


Ignore T-Lens warnings about folding layer norm into unembed, we just did it manually
torch.Size([9, 7, 7, 20])


In [18]:
s.shape
s[0].shape

torch.Size([7, 7, 20])

In [ ]:
import torch
from torch.utils.data import DataLoader
# Assuming necessary imports for your dataset and env

dataloader = DataLoader(dataset, batch_size=2048, shuffle=True)

# Create lists to collect batches
obs_list, actions_list, rtg_list, ti_list = [], [], [], []

# Iterate through the dataloader
for i, batch in enumerate(dataloader):
    s, a, r, d, rtg, ti, m = batch
    a[a == -10] = env.action_space.n
    obs, actions, rtg, ti = preprocess_inputs(dt, s, rtg, a, ti)
    
    # Append to lists
    obs_list.append(obs)
    actions_list.append(actions)
    rtg_list.append(rtg)
    ti_list.append(ti)
    
    # Break after collecting 10 batches
    if i == 9:
        break

# Concatenate along the batch dimension (assuming it's the 0-th dimension)
obs_stacked = torch.cat(obs_list, dim=0)
actions_stacked = torch.cat(actions_list, dim=0)
rtg_stacked = torch.cat(rtg_list, dim=0)
ti_stacked = torch.cat(ti_list, dim=0)

# Print shapes
print(obs_stacked.shape)
print(actions_stacked.shape)
print(rtg_stacked.shape)
print(ti_stacked.shape)


In [ ]:
import torch 

starts_with_padding_mask = (actions_stacked[:,:4] == torch.tensor([7,7,7,7])).sum(dim=1) == 4
ends_with_forwards = (actions_stacked[:,-4:] == torch.tensor([2,2,2,2])).sum(dim=1) == 4
print(ends_with_forwards.sum())


In [ ]:
from src.visualization import render_minigrid_observation, render_minigrid_observations
from src.environments.utils import reverse_one_hot
import plotly.express as px
import numpy as np 
import random

# get a random batch index
batch_index = random.randint(0, obs_stacked.shape[0] - 1)
batch_index 

one_hot_s = obs_stacked[batch_index]
original_obs = torch.from_numpy(np.stack([reverse_one_hot(one_hot_s[i]) for i in range(one_hot_s.shape[0])]))
images = render_minigrid_observations(env, original_obs)
px.imshow(images, animation_frame=0).show()

In [ ]:
import plotly.express as px 
px.histogram(dataset.traj_lens)


In [ ]:
obs, actions, rtg, ti = preprocess_inputs(dt, s, rtg, a, ti)


In [ ]:

tokens = dt.to_tokens(obs, actions.unsqueeze(-1), rtg, ti.unsqueeze(-1))
tokens.shape # now we have the tokens, we can run the model 
x, cache = dt.transformer.run_with_cache(tokens, remove_batch_dim=False)

state_preds, action_preds, reward_preds = dt.get_logits(
    x,
    batch_size=128,
    seq_length=9,
    no_actions=False,  # we always pad now.
)

action_preds

## Generate the Trajectories Manually

In [66]:
from src.environments.memory import MemoryEnv
from src.decision_transformer.offline_dataset import one_hot_encode_observation
import torch
import numpy as np
import plotly.express as px

env = MemoryEnv(
    size=7, 
    random_direction=False, 
    random_start_pos=False, 
    random_length=False, 
    render_mode="rgb_array")

env.reset()

all_observations = []
all_actions = []

for target_obj in ["key", "ball"]:
    for target_pos in ["top", "bottom"]:
        observations = [] 
        actions= []

        # generate the new environment 
        env._gen_grid(7,7,target_obj=target_obj, target_pos=target_pos)
        obs = env.gen_obs()['image']
        observations.append(obs)
        for _ in range(4):
            action = 2
            next_obs, reward, next_done, next_truncated, info  = env.step(action)
            observations.append(next_obs['image'])
            actions.append(action)

        all_observations.append(observations)
        all_actions.append(actions)
        


all_observations = torch.from_numpy(np.stack(all_observations))
all_observations.shape

torch.Size([4, 5, 7, 7, 3])

In [67]:
# to view them we can use 
from src.visualization import render_minigrid_observations
import plotly.express as px

images = torch.stack([torch.from_numpy(render_minigrid_observations(env, s)) for s in all_observations])
images.shape

px.imshow(images, animation_frame=1, facet_col=0).show()